In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras

In [2]:
file_url = 'data/data.csv'
dataframe = pd.read_csv(file_url)
print(dataframe.shape)
dataframe.head(20)


(500, 10)


In [4]:
val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

Using 400 samples for training and 100 for validation
